# Lab session 02: Artificial Neural Networks 
by Dr Ivan Olier-Caparroso, last updated: Jan-22

## Introduction
The aim with this tutorial is to get familiar with the use of the *scikit-learn* library (aka *sklearn*) in *Python*.

## Exercise 1
Write a small Python function that implements a Perceptron. The function should take two binary inputs and returns the output of the perceptron. Use the function to test the *AND* gate.




## Exercise 2
Write a small Python function that implements an MLP of the *XOR* gate. The function should take two binary inputs and returns the output of the MLP. Use the function to test the *XOR* gate.

## Exercise 3
Modify above function such that it uses ReLUs and a sigmoid activation functions of the neurons in the hidden layer and output layer, respectively. Verify whether the new architecture still implements a XOR. If not, could you try to find possible new weight and bias values?




## Exercise 4
The following tutorial explains how to implement MLPs using *scikit-learn*. The task consists in implementing a classifier to identify costumers at risk of credit card defaulting. After completing the tutorial, you will be asked to complete a similar task independently.

### Tutorial

#### Dataset

The power of the model is in its ability to predict samples that were unseen during training. That is, the model ability to generalise. Ideally, we should use an independent dataset to test the model. However, it is quite frequent that such dataset is not available. Typically, we just randomly split the dataset available into two subsets, training and test.

We are going to use a dataset to model the risk of credit card default. The dataset is called *default* and can be download from my GitHub repository by following this link: https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/default.csv. 

`default.csv` is a comma-separated value (CSV) file. There are several Python functions that can be used to read the file. We will use `read_csv` which comes with the `pandas` Python package. `pandas` is a very robust and extensive Python package which has become the standard for data manipulation in Python. 

* Download the file and put it in a location that is of easy access (it could be within the same working directory). Alternatively, you can directly access its Internet location using the Pandas's function `read_csv`

* Read the `default.csv` file:

In [ ]:
import pandas as pd
dset = pd.read_csv('https://raw.githubusercontent.com/iaolier/7021DATSCI/main/data/default.csv')

Above code reads the `default.csv` file and creates a data frame named `dset`. Data frames are the natural Python objects to represent datasets, where columns represent variables (either inputs or outputs) and rows, the samples or observations.
There are several ways to investigate the content of a data frame. If it is not too long or wide, we can just double-click on its name under the *File Browser* panel (if you are using Jupyter Lab). However, it is very common that the number of rows or columns are to big. 

In [ ]:
# Option 2 - if you are working with Google Colab and want to upload a local CSV file, uncomment and run the following code:

#from google.colab import files
#uploaded = files.upload()
#import io
#dset = pd.read_csv(io.BytesIO(uploaded['default.csv']))
#dset.head()



* We can also use the function `head()` to inspect the first 5 rows:

In [ ]:
dset.head()

From above, each row corresponds to a costumer. `default` dataset has 4 variables: 

* *default* - the output, 
* *student* - whether it is a student or not, 
* *balance* - current credit balance, and 
* *income* -yearly income (in USD).

Therefore, the task that our first neural network model will learn is to predict the risk of default as a function of the rest of the variables (inputs). In addition, notice that `default` and `student` variables are categorical (*Yes/No*). Although Python can code categorical variables with `category`, the reality is that `default` and `student` columns are `object` type. This is simply because when the data frame was created, Python could not infer their type.

In [ ]:
dset.dtypes

In order to use the categorical variables for modelling, we must convert them into numeric. We can decide that "No" is to be coded as 0, and "Yes", as 1. To do this, we can convert them to categorical and, then, to numeric.
* Below, we use `select_dtypes` to choose the `object` type columns:

In [ ]:
sel_cols = dset.select_dtypes(['object']).columns
sel_cols

* Now, the variables are converted to numeric:

In [ ]:
dset[sel_cols] = dset[sel_cols].apply(lambda x: x.astype('category').cat.codes)
dset.head()

* Now, we should put the data in the right format. That is, inputs in a matrix and outputs in a vector. To do this, we can use the function `train_test_split` which comes with the `sklearn` package:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dset.drop('default', axis=1), 
                                                    dset['default'], 
                                                    test_size=0.3, random_state=123)

In [ ]:
X_train.shape

It is recommended to scale the columns. Some ML algorithms require us to do it, whilst for others it is just a recommendation. There are also algorithms that variable scaling is simply not needed. Although there are several ways to do this, here we will use `StandardScaler` which subtracts the column means and then scales them by dividing them by their standard deviations.

In [ ]:
from sklearn import preprocessing
mm_scaler = preprocessing.StandardScaler()  # creates a Transformer (set of preprocessing operations)
X_train = mm_scaler.fit_transform(X_train)  # estimates transformation parameters using the training set, and applies to them
X_test = mm_scaler.transform(X_test) # transforms test set using training set parameters.

More information about data preprocessing using *scikit-learn* can be found here: https://scikit-learn.org/stable/modules/preprocessing.html

* You can verify the new column ranges:

In [ ]:
from scipy import stats
stats.describe(X_train)  # X_train is a numpy array, so you can't just use the pandas's describe function.

In [ ]:
stats.describe(X_test)

#### Building the first model in *Scikit-Learn*
 We will build a multi-layer perceptron (MLP) model of the `default` data using the `sklearn` library. You can find some documentation about the *Scikit-Learn* library in https://scikit-learn.org/stable/index.html. More details on how to build neural network models with *Scikit-Learn* can be found here: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. 

Let's first design the network architecture. The network will require as many inputs as input variables, and as many outputs as output variables. Therefore, our first network will have 3 inputs and 1 output. The question is whether we use hidden neurons (MLPs) or not (Perceptron). 

* Let's build our first architecture which is a multi-layer perceptron (MLP) with one hidden layer. The model will have 3 inputs (one for each input variable) and 1 output neuron (that predicts whether the costumer defaults or not). The activation function in the output should be `sigmoid` since the task is basically a binary classification problem. A `sigmoid` function will ouput the probability of having the positive class (which is *YES*, a "default"). It is less clear the shape of the hidden layer. Let's assume for now that it will be formed by 5 neurons, all with the *ReLU* activation function:

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_mdl = MLPClassifier(hidden_layer_sizes=(5,),random_state=42, max_iter=300)


We use the function `MLPClassifier` to create the model architecture. Please review its full set of parameters, default values and, in general, how it is used by folling this link: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier. The above `mlp_mdl` is a Python object that contains an MLP with one hidden layer of 5 neurons. By default, `MLPClassifier` assumes *ReLU* functions in the hidden layers and *sigmoid* functions in the output layer. It also assumes that *ADAM* as the default optimiser. Again, refer to the documentation for more details.

Now, it is time for training the model. Almost all `sklearn` model is trained using the function `fit`, which takes the training set as input.

In [ ]:
mlp_mdl.fit(X_train, y_train)

In order to predict new observations we can use the standard `sklearn` function `predict`, that predicts the class, or `predict_proba`, that predicts the class probability.

* Below, the class of the first 200 observations from the test set are predicted

In [ ]:
mlp_mdl.predict(X_test[0:200,])

* Also, the class probability of the first 10 observations:

In [ ]:
mlp_mdl.predict_proba(X_test[0:10,])

The attribute `coefs_` contains the weight values that were estimated during the training process. `coefs_` is a list of arrays:

In [ ]:
[coef.shape for coef in mlp_mdl.coefs_]

The above code confirms that we have a network with 3 inputs, 1 hidden layer with 5 neurons and 1 neuron in the output layer.

* We can also extract the weights of the connections from the inputs to the hidden layer as follows:

In [ ]:
mlp_mdl.coefs_[0]

* And, from the hidden layer to the output:

In [ ]:
mlp_mdl.coefs_[1]

Now it is time to evaluate the model. We would like to know things like its receiver operator characteristic curve (ROC), the area under its ROC (AUC-ROC or simply AUC), etc. More details about classification metrics can be found here: https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics.

As seen in prior sessions, accuracy doesn't always reflect the true quality of the model performance, particularly with unbalanced class problems. Therefore, we can perform an ROC analysis.  

**Note: the below code might not work if you have an early version of *scikit-learn*. If fails, please have a look at the code in comments.**

In [ ]:
from sklearn.metrics import RocCurveDisplay
y_pred_prob = mlp_mdl.predict_proba(X_test)
RocCurveDisplay.from_predictions(y_test, y_pred_prob[:,1])

`RocCurveDisplay.from_predictions` takes the true labels (`y_test`) and the predicted probabilities of the class 1. Note that `y_pred_prob` is a 2D matrix. The first column (i.e. column 0) contains the probabilities of being the class 0, whilst the second column (i.e. column 1), the class 1, which is the one of interest.

In [ ]:
## If above code fails due to version compatibilities, then you can try the following code:

# import matplotlib.pyplot as plt     # to generate plots

## to allow for plots in notebooks
# %matplotlib inline  

# from sklearn.metrics import roc_curve, auc
# fpr, tpr, thres = roc_curve(y_test, y_pred_prob[:,1])
# auc_score = auc(fpr, tpr)
# plt.figure()
# plt.plot(fpr, tpr, label="ROC curve (area = %0.2f)" % auc_score)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.legend(loc="lower right")
# plt.show()

If we would like to get the *AUC* (not the ROC plot), we can just use `roc_auc_score`:

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_prob[:,1])

### Question

* Repeat the same analysis but using:
1. an architecture of one hidden layer of 10 neurons.
2. an architecture of two hidden layers of 10 and 5 neurons.

## Exercise 5

Repeat Exercise 3 of the Lab Session 01 but using an MLP. Consider several MLP architectures by varying the number of layers and number of neurons per layer. Discuss the impact of the several architectures on the performance of the models.